In [1]:
import trimesh
import rospkg
from trimesh import transformations
from geometry_msgs.msg import Quaternion, Point, Pose, PoseStamped, Point32
rospack = rospkg.RosPack()

In [2]:
collision_manager = trimesh.collision.CollisionManager()

In [3]:
package_path = rospack.get_path('task_planner')

In [4]:
env_pose = PoseStamped()
env_pose.header.frame_id = "base_link"
env_pose.pose.position.x = 0.51
env_pose.pose.position.y = 0.05
env_pose.pose.position.z = 0
env_pose.pose.orientation.x = 0
env_pose.pose.orientation.y = 0
env_pose.pose.orientation.z = 0.707
env_pose.pose.orientation.w = -0.707

env_mesh = trimesh.load_mesh(package_path + '/mesh_dir/desk_with_object.stl')
env_pose_matrix = transformations.quaternion_matrix([env_pose.pose.orientation.w, 
                                                    env_pose.pose.orientation.x, 
                                                    env_pose.pose.orientation.y, 
                                                    env_pose.pose.orientation.z])
env_pose_matrix[0, 3] = env_pose.pose.position.x
env_pose_matrix[1, 3] = env_pose.pose.position.y
env_pose_matrix[2, 3] = env_pose.pose.position.z
env_mesh.apply_transform(env_pose_matrix)

<trimesh.Trimesh(vertices.shape=(2975, 3), faces.shape=(12058, 3))>

In [5]:
collision_manager.add_object('env', env_mesh)

In [6]:
list_of_feasible_object = []
num_of_row = 6
num_of_col = 10
x_shift = 0.56
y_shift = 0.0
z_shift = 0.8

for i in range(num_of_row):
    for j in range(num_of_col):
        obj_mesh = trimesh.load_mesh(package_path + '/mesh_dir/cup.stl')
        
        # generate a random pose
        obj_pose = PoseStamped()
        obj_pose.header.frame_id = "base_link"
        obj_pose.pose.position.x = i * 0.1 - num_of_row * 0.1 / 2 + x_shift
        obj_pose.pose.position.y = j * 0.1 - num_of_col * 0.1 / 2 + y_shift
        obj_pose.pose.position.z = z_shift
        obj_pose.pose.orientation.x = 0
        obj_pose.pose.orientation.y = 0
        obj_pose.pose.orientation.z = 0
        obj_pose.pose.orientation.w = 1
        
        obj_pose_matrix = transformations.quaternion_matrix([obj_pose.pose.orientation.w, 
                                                             obj_pose.pose.orientation.x, 
                                                             obj_pose.pose.orientation.y, 
                                                             obj_pose.pose.orientation.z])
        obj_pose_matrix[0, 3] = obj_pose.pose.position.x
        obj_pose_matrix[1, 3] = obj_pose.pose.position.y
        obj_pose_matrix[2, 3] = obj_pose.pose.position.z
        obj_mesh.apply_transform(obj_pose_matrix)
        
        collision_manager.add_object('obj' + str(i * num_of_col + j), obj_mesh)

        if(not collision_manager.in_collision_internal()):
            list_of_feasible_object.append(obj_mesh)

        collision_manager.remove_object('obj' + str(i * num_of_col + j))    


In [7]:
list_of_feasible_object.append(env_mesh)

In [8]:

# Create a scene
scene = trimesh.Scene(list_of_feasible_object)

coordinate_frame = trimesh.creation.axis()

scene.add_geometry(coordinate_frame)

# Show the scene
scene.show()